In [5]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [2]:
url = "https://www.uiu.ac.bd/notice/"
mongo_string = "mongodb://localhost:27017/"

In [3]:
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [4]:
notice_card = soup.find_all("div", class_="notice")
len(notice_card)

12

In [5]:
client = MongoClient(mongo_string, 27017)
db = client["uiu_world"]
db_collection = db["notice"]

In [6]:
notice_card[0]

<div class="notice">
<div class="image">
<img alt="" src=""/>
</div>
<div class="details">
<!-- date -->
<div class="date-container">
<!-- icon -->
<span class="icon">
<img alt="" src="https://www.uiu.ac.bd/wp-content/themes/uiu-main/img/calendar-days-regular.svg"/>
</span>
<!-- date -->
<span class="date">
									April 22, 2024								</span>
</div>
<div class="title">
<a href="https://www.uiu.ac.bd/notice/notice-regarding-online-classes/">
									NOTICE REGARDING ONLINE CLASSES								</a>
</div>
</div>
</div>

In [7]:
for notice in notice_card:
    date = notice.find("span", class_="date").text.strip().split(",")[0]
    title = notice.find("div", class_="title").text.strip().capitalize()
    notice_url = notice.select(".title > a")[0]["href"].strip()

    data = {"date": date, "title": title, "url": notice_url}

    # db_collection.insert_one(data)

In [19]:
# for notice in notice_card:
#     details_url = notice.select(".title > a")[0]["href"].strip()
#     print(details_url)
# response = requests.get(
#     "https://www.uiu.ac.bd/notice/schedule-to-collect-original-certificates-all-graduates-eligible-for-7th-convocation-2024/"
# )
# soup = BeautifulSoup(response.text, "html.parser")
# data_list = soup.find("div", class_="notice-details").find_all("p")
# for i in data_list:
#     print(i.text.strip())

In [45]:
for notice in notice_card:
    res_details = requests.get(notice_card[3].select(".title > a")[0]["href"].strip())
    soup = BeautifulSoup(res_details.text, "html.parser")
    data_list = soup.find("div", class_="notice-details").text.strip().split("\n")
    data_list = [i for i in data_list if i != ""]
    data_list

['United International University will remain closed on March 26, 2024 (Tuesday) on the occasion of ‘INDEPENDENCE & NATIONAL DAY’.',
 'Dr. Md. Zulfiqur RahmanRegistrar']

In [51]:
for doc in db_collection.find():
    res_details = requests.get(doc["url"])
    soup = BeautifulSoup(res_details.text, "html.parser")
    data_list = soup.find("div", class_="notice-details").text.strip().split("\n")
    data_list = [i for i in data_list if i != ""]

    db_collection.update_one({"_id": doc["_id"]}, {"$set": {"details": data_list}})

<h1>Researches<h1>

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pickle

In [2]:
url = "https://www.uiu.ac.bd/research/journal-paper/"

In [9]:
def scrape_paper(url, output_file_path):
    options = Options()
    options.add_argument("--headless")

    driver = Chrome(options=options)
    driver.get(url)

    # Wait until the specific element is visible
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located(
            (By.XPATH, '//*[@id="conference-paper"]/div[1]')
        )
    )

    response = driver.find_element(
        By.XPATH, '//*[@id="conference-paper"]'
    ).get_attribute("outerHTML")
    driver.quit()

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response, "html.parser")
    with open(f"{output_file_path}.pkl", "wb") as file:
        pickle.dump(soup, file)

In [10]:
scrape_paper(url, "journal")

In [12]:
with open("journal.pkl", "rb") as file:
    soup = pickle.load(file)

In [56]:
paper_list = []
paper_details = soup.select("#conference-paper > div")

for paper in paper_details:
    year = paper.select("span")[0].text.strip()
    paper_url = paper.select_one("a").get("href")
    publication = (
        paper.select_one(".paper-event").text.strip().split("Publication:")[1].strip()
    )
    authors = [i.text.strip() for i in paper.select(".paper-contributors span")]
    paper_tags = [i.text.strip() for i in paper.select(".paper-tags > span")]

    paper_info = {
        "year": year,
        "paper_url": paper_url,
        "publication": publication,
        "authors": authors,
        "paper_tags": paper_tags,
    }

    paper_list.append(paper_info)